# Mesa Hello World - Google Colab Compatible

This notebook demonstrates Mesa agent-based modeling in Google Colab.

In [ ]:
# Install Mesa for Google Colab
!pip install -q mesa matplotlib

In [ ]:
from mesa import Agent, Model

# Define a simple agent that prints hello
class HelloAgent(Agent):
    def __init__(self, model, name):
        super().__init__(model)
        self.name = name
    
    def step(self):
        print(f"Hello from agent {self.name} (ID: {self.unique_id})!")

# Define a simple model
class HelloModel(Model):
    def __init__(self, n_agents):
        super().__init__()
        
        # Create agents - they're automatically added to self.agents
        for i in range(n_agents):
            agent = HelloAgent(self, f"Agent-{i}")
    
    def step(self):
        # Use AgentSet instead of scheduler
        self.agents.shuffle_do("step")

In [ ]:
# Create and run the model
model = HelloModel(n_agents=5)

print("Running 3 steps of the simulation:\n")
for i in range(3):
    print(f"--- Step {i+1} ---")
    model.step()
    print()

## What's happening?

1. We defined a `HelloAgent` that prints a message when it acts
2. We created a `HelloModel` that manages multiple agents
3. The model uses `RandomActivation` to activate agents in random order each step
4. Each step, all agents say hello in a randomized sequence

This is the foundation for more complex multi-agent systems!

# Animated Visualization

Let's create a simple visualization with agents moving randomly on a grid.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mesa.space import MultiGrid
from IPython.display import HTML
import random

# Define an agent that moves randomly on a grid
class WanderingAgent(Agent):
    def __init__(self, model):
        super().__init__(model)
        self.color = (random.random(), random.random(), random.random())
    
    def step(self):
        # Move to a random neighboring cell
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False
        )
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

# Define a model with a grid
class GridModel(Model):
    def __init__(self, n_agents, width, height):
        super().__init__()
        self.grid = MultiGrid(width, height, torus=True)
        
        # Create agents and place them randomly on the grid
        for i in range(n_agents):
            agent = WanderingAgent(self)
            
            # Place agent at random position
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(agent, (x, y))
    
    def step(self):
        # Use AgentSet instead of scheduler
        self.agents.shuffle_do("step")

In [ ]:
# Create the model
grid_model = GridModel(n_agents=10, width=20, height=20)

# Create figure for animation
fig, ax = plt.subplots(figsize=(8, 8))

def draw_grid(step_num):
    ax.clear()
    ax.set_xlim(0, grid_model.grid.width)
    ax.set_ylim(0, grid_model.grid.height)
    ax.set_aspect('equal')
    ax.set_title(f'Step {step_num}')
    ax.grid(True, alpha=0.3)
    
    # Draw each agent
    for agent in grid_model.agents:
        x, y = agent.pos
        ax.scatter(x + 0.5, y + 0.5, s=300, c=[agent.color], marker='o', edgecolors='black', linewidths=2)
    
    return ax.patches

def animate(frame):
    if frame > 0:
        grid_model.step()
    return draw_grid(frame)

# Create animation
anim = animation.FuncAnimation(fig, animate, frames=30, interval=200, blit=False, repeat=True)

# Display the animation
plt.close()  # Prevent static plot from showing
HTML(anim.to_jshtml())

This creates an interactive animation showing 10 agents (colored dots) wandering randomly on a 20×20 grid. Each agent moves to a random neighboring cell at each step. The animation runs for 30 steps and loops continuously.

# Interactive Simulation (Colab Compatible)

Google Colab has limited support for ipywidgets, so we use a simpler approach with automatic animation.

In [ ]:
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import time

# Create a new model for interactive simulation
interactive_model = GridModel(n_agents=15, width=20, height=20)
step_count = 0

def draw_current_state():
    global step_count
    clear_output(wait=True)
    
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xlim(0, interactive_model.grid.width)
    ax.set_ylim(0, interactive_model.grid.height)
    ax.set_aspect('equal')
    ax.set_title(f'Step {step_count}')
    ax.grid(True, alpha=0.3)
    
    # Draw each agent
    for agent in interactive_model.agents:
        x, y = agent.pos
        ax.scatter(x + 0.5, y + 0.5, s=300, c=[agent.color], 
                  marker='o', edgecolors='black', linewidths=2)
    
    plt.tight_layout()
    plt.show()

# Run animation with live updates
print("Running simulation...")
for i in range(30):
    draw_current_state()
    interactive_model.step()
    step_count += 1
    time.sleep(0.3)  # Pause between frames

print("\nSimulation complete!")

This version automatically runs through 30 steps with a 0.3 second delay between frames. Change the range and sleep values to adjust the animation speed and duration!